In [1]:
import numpy as np

from gradvi.models.linear_model import LinearModel
from gradvi.tests import toy_priors

In [2]:
def get_lm_data(n = 100, p = 200, p_causal = 50, pve = 0.5, rho = 0.4):

    def sd2_from_pve (X, b, pve):
        return np.var(np.dot(X, b)) * (1 - pve) / pve

    np.random.seed(100)

    """
    Equicorr predictors
    X is sampled from a multivariate normal, with covariance matrix V.
    V has unit diagonal entries and constant off-diagonal entries rho.
    """
    iidX    = np.random.normal(size = n * p).reshape(n, p)
    comR    = np.random.normal(size = n).reshape(n, 1)
    X       = comR * np.sqrt(rho) + iidX * np.sqrt(1 - rho)
    bidx    = np.random.choice(p, p_causal, replace = False)
    b       = np.zeros(p)
    b[bidx] = np.random.normal(size = p_causal)
    s2      = sd2_from_pve(X, b, pve)
    y       = np.dot(X, b) + np.sqrt(s2) * np.random.normal(size = n)
    return X, y, b, s2

In [44]:
X, y, b, s2 = get_lm_data()
priors = toy_priors.get_all()
prior = priors[1]
objtype = "reparametrize"

lm = LinearModel(X, y, b, s2, prior, objtype = objtype)
eps = 1e-8

In [45]:
lm.objective

-64.55842827506467

In [46]:
d1 = lm.s2grad
lm_eps = LinearModel(X, y, b, s2 + eps, prior, objtype = lm._objtype)
d2 = (lm_eps.objective - lm.objective) / eps

assert(np.allclose(d1, d2, atol = 0.1, rtol = 1e-8))

In [49]:
d1 - d2

-4.369804551540568e-06

In [48]:
d2

-0.4552106247501797

In [38]:
d1 = prior.wmod_grad(lm.wgrad)
d2 = np.zeros(prior.k)
for i in range(prior.k):
    amod_eps = prior.wmod.copy()
    amod_eps[i] += eps
    prior_eps = toy_priors.get_from_same_class(prior, prior.w)
    prior_eps.update_wmod(amod_eps)
    lm_eps = LinearModel(X, y, b, s2, prior_eps, objtype = lm._objtype)
    d2[i] = (lm_eps.objective - lm.objective) / eps 
assert(np.allclose(d1, d2, atol = 0.1, rtol = 1e-8))

In [39]:
d2[np.where(np.abs(d1 - d2) > 5e-6)]

array([-39.28051789,   1.28738326,   6.22167889,   9.03039401,
        10.80410248,  11.98969035])

In [40]:
d1[np.where(np.abs(d1 - d2) > 5e-6)]

array([-39.29415196,   1.28257268,   6.21461043,   9.02204136,
        10.79494099,  11.97998651])

In [41]:
np.max(np.abs(d1 - d2))

0.013634078388456317

In [42]:
d1

array([-39.29415196,   1.28257268,   6.21461043,   9.02204136,
        10.79494099,  11.97998651])

In [43]:
d2

array([-39.28051789,   1.28738326,   6.22167889,   9.03039401,
        10.80410248,  11.98969035])

In [11]:
s2

29.04939967335439

In [15]:
prior.is_scaled

False

In [5]:
for i in range(prior.k):
    w_eps     = prior.w.copy()
    w_eps[i] += eps
    prior_eps    = toy_priors.get_from_same_class(prior, w_eps)
    lm_eps = LinearModel(X, y, b, s2, prior_eps, objtype = lm._objtype)
    d1 = lm.wgrad[i]
    d2 = (lm_eps.objective - lm.objective) / eps
    assert(np.allclose(d1, d2, atol = 1e-4, rtol = 1e-8))

In [6]:
wk = prior.w.copy()
akjac  = np.log(np.exp(1)) * wk.reshape(-1, 1) * (np.eye(prior.k) - wk)
agrad  = np.sum(lm.wgrad * akjac, axis = 1)
ak = prior.wmod.copy()

In [7]:
agrad

array([-8.85627423, -0.84186851,  0.14437774,  1.55167745,  3.20918958,
        4.79289797])

In [8]:
prior.wmod_grad(lm.wgrad)

array([-8.85627423, -0.84186851,  0.14437774,  1.55167745,  3.20918958,
        4.79289797])

In [9]:
eps2 = 1e-3
agrad = prior.wmod_grad(lm.wgrad)
for i in range(prior.k):
    ak_eps = prior.wmod.copy()
    ak_eps[i] += eps2
    prior_eps = toy_priors.get_from_same_class(prior, wk.copy())
    prior_eps.update_wmod(ak_eps)
    lm_eps = LinearModel(X, y, b, s2, prior_eps, objtype = lm._objtype)
    d1 = agrad[i]
    d2 = (lm_eps.objective - lm.objective) / eps2
    assert(np.allclose(d1, d2, atol = 1e-2, rtol = 1e-8))

In [10]:
lm_eps = LinearModel(X, y, b, s2 + eps2, prior, objtype = lm._objtype)
d1 = lm.bgrad[i]
d2 = (lm_eps.objective - lm.objective) / eps2
assert(np.allclose(d1, d2, atol = 1e-4, rtol = 1e-8))

AssertionError: 

In [11]:
d1

-0.21526161256059104

In [12]:
d2

-1.4024921513566824